In [21]:
import polars as pl
from glob import glob
import os 
from os.path import join, exists, splitext, abspath
from pathlib import Path
import pandas as pd
import re
from shutil import copy

In [2]:
ROOTDIR = "../COVID-CT"
exists(ROOTDIR)

True

# csvの作成
1 -> covid  
0 -> non covid

In [22]:
covid_df = pd.read_excel(join(ROOTDIR, "COVID-CT-MetaInfo.xlsx"))
non_df = pd.read_csv(join(ROOTDIR, "NonCOVID-CT-MetaInfo.csv"))

In [23]:
covid_file_name_list  = pl.DataFrame(covid_df[["File name"]].dropna()).to_series().to_list()
non_file_name_list  = pl.DataFrame(non_df[["image name"]].dropna()).to_series().to_list()


In [24]:
covid_df_with_label = pl.DataFrame([covid_file_name_list, [1 for i in range(len(covid_file_name_list))]], ["file_name", "target"])
non_df_with_label = pl.DataFrame([non_file_name_list, [0 for i in range(len(non_file_name_list))]], ["file_name", "target"])


In [25]:
covid_df_with_label = covid_df_with_label.with_columns(file_name = pl.when(pl.col("file_name").str.contains(".png")).then(pl.col("file_name")).otherwise(pl.col("file_name")+".png"))

In [26]:
data_df = pl.concat([covid_df_with_label, non_df_with_label], how="vertical")

In [27]:
with open(join(ROOTDIR,"Data-split", "COVID", "testCT_COVID.txt"), "r") as f:
    lines = f.readlines()
covid_test = [line.strip() for line in lines]

with open(join(ROOTDIR,"Data-split", "COVID", "trainCT_COVID.txt"), "r") as f:
    lines = f.readlines()
covid_train = [line.strip() for line in lines]

with open(join(ROOTDIR,"Data-split", "COVID", "valCT_COVID.txt"), "r") as f:
    lines = f.readlines()
covid_val = [line.strip() for line in lines]

with open(join(ROOTDIR,"Data-split", "NonCOVID", "testCT_NonCOVID.txt"), "r") as f:
    lines = f.readlines()
non_test = [line.strip() for line in lines]

with open(join(ROOTDIR,"Data-split", "NonCOVID", "trainCT_NonCOVID.txt"), "r") as f:
    lines = f.readlines()
non_train = [line.strip() for line in lines]

with open(join(ROOTDIR,"Data-split", "NonCOVID", "valCT_NonCOVID.txt"), "r") as f:
    lines = f.readlines()
non_val = [line.strip() for line in lines]


In [28]:
test_file_name_list = covid_test + non_test
train_file_name_list = covid_train + non_train
val_file_name_list = covid_val + non_val

In [29]:
def get_mode(file_name: str)->str:
    if file_name in train_file_name_list:
        return "train"
    elif file_name in test_file_name_list:
        return "test"
    elif file_name in val_file_name_list:
        return "val"
    else:
        return "no_mode"

In [30]:
data_df = data_df.with_columns(mode=pl.col("file_name").map_elements(lambda x: get_mode(x)))

In [31]:
data_df = data_df.with_columns(
    path = pl.when(
        pl.col("target")==1) \
            .then(join(ROOTDIR, "Images-processed", "CT_COVID/")+pl.col("file_name")) \
            .otherwise(join(ROOTDIR, "Images-processed", "CT_NonCOVID/")+pl.col("file_name"))
            )

In [32]:
for path in data_df["path"].to_list():
    if exists(path)==False:
        print(path)

In [33]:
names = []
for path in data_df["path"].to_list():
    name = Path(path).name
    names.append(name)

In [34]:
len(names), len(set(names))

(746, 746)

In [35]:
for path in data_df["path"].to_list():
    new_path = path.replace("COVID-CT/Images-processed/CT_COVID", "COVID/images").replace("COVID-CT/Images-processed/CT_NonCOVID", "COVID/images")
    copy(src=path, dst=new_path)

In [36]:
data_df = data_df.with_columns(new_path=pl.col("path").map_elements(
    lambda x: abspath(x.replace("COVID-CT/Images-processed/CT_COVID", "COVID/images").replace("COVID-CT/Images-processed/CT_NonCOVID", "COVID/images"))))

In [37]:
data_df = data_df.drop("path").with_columns(pl.col("new_path").alias("path")).drop("new_path")


In [38]:
for path in data_df["path"].to_list():
    if exists(path)==False:
        print(path)

In [40]:
data_df.write_csv("../covid_data.csv")